In [7]:
#last_pcr=5
#05.27.21
last_pcr=14

In [8]:
import pandas as pd
import numpy as np
import re
import datetime as dt

In [9]:
path='C:\\Users\\camendol\\UHG\\Client PCR Intake - General\\'
file='PCR Intake Form - Responses.xlsx'
file_path=path+file

In [10]:
#Open connection to EXCEL
xl_file = pd.ExcelFile(file_path)
xl_file.sheet_names 

['Form1', '_56F9DC9755BA473782653E2940F9']

In [11]:
#Read a tab
pcr_data=xl_file.parse('Form1')

In [12]:
curr_pcr=last_pcr+1
# Current Row to dictionary
current=pcr_data.loc[pcr_data['ID']==last_pcr+1].to_dict('records')[0]

In [13]:
# for col in current:
#     print(col)
#     print('\t',str(current[col]),'\n')
current

{'ID': 15,
 'Start time': Timestamp('2021-05-24 12:41:03'),
 'Completion time': Timestamp('2021-05-24 12:45:51'),
 'Email': 'anonymous',
 'Name': nan,
 'Client Name': 'Canopy Health',
 'Client Contact for Change - Name, Email, and Phone': 'Paolo Nunes, 510.254.4662, Paulo.Nunes@CanopyHealth.com',
 'Optum Point of Contact - Name, Email, and Phone': 'Lillian Dittrick, lillian.dittrick@optum.com',
 'Other Optum Stakeholders - Name, Email, and Phone': 'Sean Abramson, abramsos@advisory.com',
 'Is this work billable to client?  If yes, requires a formal LOE which takes 5 business days to complete.': 'No',
 'Start Date for Monthly Production Run Cycle': nan,
 'Client Status': 'Production',
 'Implemented Optum Products': 'IPro;IIRP;',
 'Client Type': 'Provider;',
 'Describe the current condition\nExample:  Client currently runs IPro with 1 year data, and ACCOUNT field currently unpopulated': 'Client current has IIRP with a one-month runout period.',
 'What change is requested to the current co

In [14]:
title='Production Change Request: #'+str(current['ID'])+' '+current['Client Name']
submitted='Submitted: '+str(current['Completion time'])
#contact=current['Name']+' - '+current['Email']
contact=current['Optum Point of Contact - Name, Email, and Phone']
cli_spec=current['Client Type']+'-'+current['Client Status']+' '+current['Implemented Optum Products']
curr_state=current['Describe the current condition\nExample:  Client currently runs IPro with 1 year data, and ACCOUNT field currently unpopulated']
req_chg=current['What change is requested to the current condition? \nExample:  Expand to 24 Months of Data for Claims and Eligibility, and account field change to GROUP_NUM. \n-ACCOUNT to be populated with Group_nu...']
aff_del=current['What existing deliverables would this change impact? \nExample:\nIPro Datamart, and client extracts.']
add_comm=current['Additional Comments (Known Constraints, Risks, Delays in Submission, Requests to expedite)']
estimate='This is a billable activity/Estimate is required.'
if current['Is this work billable to client?  If yes, requires a formal LOE which takes 5 business days to complete.']!='Yes':
    estimate='This IS NOT a billable activity/Estimate IS NOT required.'
if current['Please enter start date of monthly production cycle run (Enter "Unknown" if applicable)'].notnull():  
    

In [ ]:
#HTML
#Create Output DF
index=[ 'Contact Info'
       ,'Client Specifics'
       ,'Current State'
       ,'Requested Changes'
       ,'Affected Deliverables'
       ,'Additional Commentary'
       ,'Billable/Estimate']
a = { 'Current': { 'Contact Info':contact
     ,'Client Specifics':cli_spec
     ,'Current State':curr_state
     ,'Requested Changes':req_chg
     ,'Affected Deliverables':aff_del
     ,'Additional Commentary':add_comm
     ,'Billable/Estimate':estimate}
    }
     
df = pd.DataFrame(data=a,index=index).reset_index()

pd.set_option('colheader_justify', 'center')   # FOR TABLE <th>

html_string = '''
<html>
  <head><title>'''+title+'''</title> 
  <style> 
  table, th, td {{font-size:14pt; 
                  border:1px solid black; 
                  border-collapse:collapse; 
                  text-align:left;
                  width:50%
                  }}
  th, td {{padding: 5px;}}
  </style>
  </head>
  <body>
    <h1 style="text-align:left">'''+title+'''</h1>
    <h1 style="text-align:left">'''+submitted+'''</h1>
    {table}
  </body>
</html>.
'''
final_html='C:\\Users\\camendol\\Documents\\PCRs\\Teams_Auto\\PCR_'+str(current['ID'])+'.html'
# OUTPUT AN HTML FILE
with open(final_html, 'w') as f:
    f.write(html_string.format(table=df.to_html(classes='mystyle',index=False,header=False)))

In [ ]:
path='C:\\Users\\camendol\\Downloads\\'
file='UserStoryImportTemplate (2).csv'

file_path=path+file

In [ ]:
#Open connection to EXCEL
templ_file = pd.read_csv(file_path) 

In [ ]:
#templ_file

In [ ]:
card_raw=templ_file.loc[templ_file.index==0].to_dict('records')[0]

In [ ]:
#card_raw

In [ ]:
for item in card_raw:
    card_raw[item]=''
    
card_raw['Expedite']=False
card_raw['Display Color']='#323232'
card_raw['Blocked Reason']=''
card_raw['Blocked']=False
card_raw['Name']=title+submitted
card_raw['Description']='??'
card_raw['Owner']=''
card_raw['Notes']='note'
card_raw['Impact Kanban State']='Requests'
card_raw['Account Manager']=''
card_raw['Priority']='Normal'
#card_raw['Description']=curr_state+'  --{CR}--  '+req_chg+' --{CR}-- Affected Deliverables: '+aff_del+'---{CR}---'+str(add_comm)
card_raw['Description']=html_string.format(table=df.to_html(classes='mystyle',index=False,header=False))
card_raw['Support Issue ID']='PCR#'+str(current['ID'])
#card_raw

In [ ]:
final_card = pd.DataFrame.from_dict(card_raw, orient='index').T

In [ ]:
final_card.to_csv('C:\\Users\\camendol\\Downloads\\test_card.csv')